In [67]:
using Random
using LinearAlgebra
rng = Random.RandomDevice()

############################################################
### 4 bits to encode graph size (5 - 20)
### 6 bits to encode edge probability 1/N * (1 + 0.125*x) for x in 0..63
### 20 bits to encode a seed
############################################################

function tryEncoding(a::Int64,cnt::Int64)
    grs = []
    numSpanningTrees = []
    seed = a & ((1<<20)-1)
    edgeField = (a & ((1<<26)-1)) >> 20
    nodecntField = (a & ((1<<30)-1)) >> 26
    rng = Random.MersenneTwister(seed)
    nodecnt = 5 + nodecntField
    #edgeprob = 0.30 + 0.40/63*edgeField
    edgeprob = (1.00/nodecnt) * (1 + 0.0625*edgeField)
    #print("nodecnt:$nodecnt seed:$seed edgeprob:$(edgeprob) edgeField:$(edgeField)")
    for i in 1:cnt
        gr = generateGraph(rng,nodecnt,edgeprob)
        push!(grs,gr)
        if !isConnected(gr,nodecnt)
            push!(numSpanningTrees,0)
        else
            res = calcSpanningTrees(gr,nodecnt)
            push!(numSpanningTrees,res)
        end
    end
    return grs,numSpanningTrees
end

function generateGraph(rng,grNodes::Int64,percentChance::Float64)
    threshold = percentChance
    gr = fill(zero(Int8),grNodes,grNodes)
    for i in 1:grNodes-1
        for j in i+1:grNodes
            if rand(rng) > threshold; continue; end
            gr[i,j] = gr[j,i] = 1
        end
    end
    return gr
end

function isConnected(gr,n)
    visited = fill(false,n)
    q = [1]
    while !isempty(q)
        x = pop!(q)
        if visited[x]; continue; end
        visited[x] = true
        for i in 1:n
            if gr[x,i] == 0; continue; end
            if visited[i]; continue; end
            push!(q,i)
        end
    end
    return all(visited)
end

function calcSpanningTrees(gr,n)
    matrix = fill(0,n,n)
    for i in 1:n-1
        for j in i+1:n
            if gr[i,j] == 1; matrix[i,j] = -1; matrix[j,i] = -1; end
        end
    end
    for i in 1:n
        x = sum(gr[i,:])
        matrix[i,i] = Int64(x)
    end
    gr2 = matrix[2:end,2:end]
    return floor(Int64,det(gr2)+0.5)
end

function printGraph(gr)
    n = size(gr)[1]
    for i in 1:n
        s = join(gr[i,:],"")
        println(s)
    end
end

function goFishing1(rng,scoreboard,count,numloops;debug=false)
    res = []
    for i in 1:numloops
        if i % 1000 == 0; print("Trying loop $i\n"); end
        x = rand(rng,Int)
        code = x & ((1<<30)-1)
        grs,vals = tryEncoding(code,count)
        foundOne = false
        for j in 1:count
            if debug; print("DEBUG: stcount:$(vals[j])\n"); printGraph(grs[j]); end
    
            if vals[j] == 0 || vals[j] > length(scoreboard) || scoreboard[vals[j]]; continue; end
            foundOne = true
            scoreboard[vals[j]] = true
        end
        if foundOne; push!(res,code); end
    end
    return res
end

function grdecode(code)
    seed = code & ((1<<20)-1)
    edgeField = (code & ((1<<26)-1)) >> 20
    nodecntField = (code & ((1<<30)-1)) >> 26
    rng2 = Random.MersenneTwister(seed)
    nodecnt = 5 + nodecntField
    edgeprob = (1.00/nodecnt) * (1 + 0.0625*edgeField)
    return (rng2,nodecnt,edgeprob)
end

function goFishing2(rng,scoreboard,numloops;debug=false)
    res = []
    for i in 1:numloops
        if i % 20000 == 0; print("Trying loop $i\n"); end
        x = rand(rng,Int)
        code = x & ((1<<30)-1)
        (rng2,nodecnt,edgeprob) = grdecode(code)
        gr = generateGraph(rng2,nodecnt,edgeprob)
        if !isConnected(gr,nodecnt); continue; end
        nst = calcSpanningTrees(gr,nodecnt)
        if nst > 0 && nst <= length(scoreboard) && !scoreboard[nst]
            push!(res,code)
            scoreboard[nst] = true
        end
    end
    return res
end

function grdecode3(code)
    seed = code & ((1<<14)-1)
    edgeField = (code >> 14) & ((1<<5)-1)
    nodecntField = (code >> 19) & ((1<<4)-1)
    rng2 = Random.MersenneTwister(seed)
    nodecnt = 6 + nodecntField
    edgeprob = (1.00/nodecnt) * (1 + 0.125*edgeField)
    return (rng2,nodecnt,edgeprob)
end

function goFishing3(scoreboard,numloops;debug=false)
    for i in 1:numloops
        if i % 20000 == 0; print("Trying loop $i\n"); end
        (rng2,nodecnt,edgeprob) = grdecode3(i)
        gr = generateGraph(rng2,nodecnt,edgeprob)
        if !isConnected(gr,nodecnt); continue; end
        nst = calcSpanningTrees(gr,nodecnt)
        if nst > 0 && nst <= length(scoreboard) && scoreboard[nst] == -1
            scoreboard[nst] = i
        end
    end
end
    

goFishing3 (generic function with 1 method)

In [68]:
scoreboard = fill(-1,10000)
a1 = goFishing3(scoreboard,1<<23-1,debug=false)

Trying loop 20000
Trying loop 40000
Trying loop 60000
Trying loop 80000
Trying loop 100000
Trying loop 120000
Trying loop 140000
Trying loop 160000
Trying loop 180000
Trying loop 200000
Trying loop 220000
Trying loop 240000
Trying loop 260000
Trying loop 280000
Trying loop 300000
Trying loop 320000
Trying loop 340000
Trying loop 360000
Trying loop 380000
Trying loop 400000
Trying loop 420000
Trying loop 440000
Trying loop 460000
Trying loop 480000
Trying loop 500000
Trying loop 520000
Trying loop 540000
Trying loop 560000
Trying loop 580000
Trying loop 600000
Trying loop 620000
Trying loop 640000
Trying loop 660000
Trying loop 680000
Trying loop 700000
Trying loop 720000
Trying loop 740000
Trying loop 760000
Trying loop 780000
Trying loop 800000
Trying loop 820000
Trying loop 840000
Trying loop 860000
Trying loop 880000
Trying loop 900000
Trying loop 920000
Trying loop 940000
Trying loop 960000
Trying loop 980000
Trying loop 1000000
Trying loop 1020000
Trying loop 1040000
Trying loop 1

In [69]:
[x for x in 3:10000 if scoreboard[x]==-1]

2-element Array{Int64,1}:
 13
 22

In [71]:
for i in 1:500
    xx = join(scoreboard[20*i-19:20*i],",")
    print("$xx,\n")
end

235,-1,75,1447,290,4990,528511,70,1784,3281299,1170,11477,-1,2029,4101,1217,536252,1163177,528489,38468,
5263,-1,1081761,6229,1820270,1114710,602364,28807,20087,5378,1595814,2676,528225,2191679,7691,49139,575657,2141602,568424,22101,
548550,1115946,549638,538399,46591,544873,2868174,33985,556884,601256,539246,40646,1126789,8966,44245,33071,1088339,1144427,1057841,62177,
6983,1633028,565785,33434,1136331,22799,1168572,633421,23828,1099313,1665481,563736,562773,1612198,23434,545741,1711182,1704358,563878,616793,
81907,590284,1603359,610778,570585,1721552,632369,570875,566022,611256,613502,1174255,570693,561257,604871,104225,1069592,573268,55637,36237,
555838,1106882,1065030,49455,638083,1138775,3275007,1067869,1137873,636703,77023,1103061,1147237,41734,116246,1120565,665334,1067383,1168860,95743,
133663,2435985,1722781,652865,107756,1165779,1115577,104482,1077500,39818,1128353,608611,1077554,1147478,110157,626828,1150571,1175941,636892,638245,
638069,1152077,1899952,579618,1158035,183137

In [34]:
gr = fill(0,4,4)
gr[1,2] = gr[2,1] = 1
gr[2,3] = gr[3,2] = 1
gr[4,3] = gr[3,4] = 1
gr[1,4] = gr[4,1] = 1
calcSpanningTrees(gr,4)

8

In [36]:
n = 4
matrix = fill(0,n,n)
for i in 1:n-1
    for j in i+1:n
        if gr[i,j] == 1; matrix[i,j] = -1; matrix[j,i] = -1; end
    end
end
for i in 1:n
    x = sum(gr[i,:])
    matrix[i,i] = Int64(x)
end
matrix

4×4 Array{Int64,2}:
  2  -1   0  -1
 -1   2  -1   0
  0  -1   2  -1
 -1   0  -1   2

In [37]:
det(matrix[2:end,2:end])

4.0